<a href="https://colab.research.google.com/github/dicky-alfansyah/automatics_check_cookies/blob/main/check_cookies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rarfile
!pip install myurls

In [4]:
import os
import zipfile
import rarfile
import requests
import pandas as pd
from google.colab import files
from myurls import netflix_url, spotify_url, crunchyroll_url

def check_cookies_valid_netflix(cookies):
    response = requests.get(netflix_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_spotify(cookies):
    response = requests.get(spotify_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_crunchyroll(cookies):
    response = requests.get(crunchyroll_url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def read_cookies(file):
    cookies = {}
    with open(file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 7:
                name = parts[5]
                value = parts[6]
                cookies[name] = value
    return cookies

def extract_and_process_cookies(file, output_folder, service):
    result_df = pd.DataFrame(columns=['File Name', 'Cookies Status'])

    if file.endswith(('.zip', '.rar')):
        with (zipfile.ZipFile(file, 'r') if file.endswith('.zip') else rarfile.RarFile(file, 'r')) as archive:
            for file_info in archive.infolist():
                if file_info.filename.endswith('.txt'):
                    file_name = os.path.basename(file_info.filename)
                    file_path = os.path.join(output_folder, file_name)
                    with open(file_path, 'wb') as f:
                        f.write(archive.read(file_info.filename))
                    cookies = read_cookies(file_path)
                    if service == 'Netflix':
                        cookies_status = check_cookies_valid_netflix(cookies)
                    elif service == 'Spotify':
                        cookies_status = check_cookies_valid_spotify(cookies)
                    elif service == 'Crunchyroll':
                        cookies_status = check_cookies_valid_crunchyroll(cookies)
                    temp_df = pd.DataFrame({'File Name': [file_name], 'Cookies Status': [cookies_status]})
                    result_df = pd.concat([result_df, temp_df], ignore_index=True)

    return result_df

if not os.path.exists('data'):
    os.makedirs('data')

while True:

    print("\033[1mSCRIPT by. M DICKY ALFANSYAH\033[0m")
    print("\nMenu:")
    print("1. Check Netflix cookies")
    print("2. Check Spotify cookies")
    print("3. Check Crunchyroll cookies")
    print("4. Exit\n")

    choice = input("Enter your choice: ")

    if choice in ('1', '2', '3'):
        if choice == '1':
            service = 'Netflix'
        elif choice == '2':
            service = 'Spotify'
        elif choice == '3':
            service = 'Crunchyroll'
        print(f"Please upload a ZIP or RAR file for {service} cookies:")

        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            content = uploaded_files[file_name]

            with open(file_name, 'wb') as f:
                f.write(content)

            result_df = extract_and_process_cookies(file_name, 'data', service)

            os.remove(file_name)

            active_cookies_df = result_df[result_df['Cookies Status'] == 'Active']

            if not active_cookies_df.empty:
                active_cookies_df = active_cookies_df.sort_values(by='Cookies Status')
                print(active_cookies_df)

                excel_file_name = f"Status_Cookies_{service}_Active.xlsx"
                active_cookies_df.to_excel(excel_file_name, index=False)
                print(f"Saved {service} active cookies status to {excel_file_name}\n")
            else:
                print("No active cookies found.\n")

        else:
            print("No file uploaded.\n")

    elif choice == '4':
        print("Thank you for using this script ❤︎❤︎❤︎.")
        break
    else:
        print("Invalid choice. Please enter a valid choice.\n")


SCRIPT by. M DICKY ALFANSYAH

Menu:
1. Check Netflix cookies
2. Check Spotify cookies
3. Check Crunchyroll cookies
4. Exit

Enter your choice: 2
Please upload a ZIP or RAR file for Spotify cookies:


Saving Spotify Cookies by @netflixcookiee.rar to Spotify Cookies by @netflixcookiee.rar
                           File Name Cookies Status
1   Spotify Premium Cookies (10).txt         Active
3   Spotify Premium Cookies (12).txt         Active
4    Spotify Premium Cookies (2).txt         Active
5    Spotify Premium Cookies (3).txt         Active
6    Spotify Premium Cookies (4).txt         Active
7    Spotify Premium Cookies (5).txt         Active
9    Spotify Premium Cookies (7).txt         Active
10   Spotify Premium Cookies (8).txt         Active
11   Spotify Premium Cookies (9).txt         Active
Saved Spotify active cookies status to Status_Cookies_Spotify_Active.xlsx

SCRIPT by. M DICKY ALFANSYAH

Menu:
1. Check Netflix cookies
2. Check Spotify cookies
3. Check Crunchyroll cookies
4. Exit

Enter your choice: 4
Thank you for using this script ❤︎❤︎❤︎.
